In [1]:
import shutil
from subprocess import check_output
from pathlib import Path
import numpy as np
import os
#import matplotlib.pyplot as plt
import pandas as pd
#from shapely.geometry import Polygon,
import RClib as RClib



In [5]:
%load_ext autoreload
%autoreload 2
''
config = RClib.loadconfigs('.\config_gareus.json')
scandf = RClib.provide_scandf(inputdirectory=config['workspace'], imageformat = config['baseimageformat'])



print(scandf.columns)

scandf= scandf.apply(RClib.defineRawTherapeeOutput, foldername='02_TIF', axis=1)
scandf= scandf.apply(RClib.defineRealityCaptureOutput, foldername='03_RC', axis=1)
#rcsettings_template = pd.read_csv('.\\allRCsettings_template.csv', sep=';', header=0) 
#print(rcsettings_template)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Index(['id', 'processingstate', 'scan_dir', 'pp3file', 'gcpsfile', 'imagedf'], dtype='object')


In [6]:

scandf2=pd.DataFrame()

for index,scan in scandf.iterrows():
    if not scan['imagedf'] is None and config['developimages']:
        if 'pp3file' in scan.keys() and scan['pp3file']:    
            scan['imagedf'] = scan['imagedf'].apply(RClib.developwithRawTherapee, pp3filepath=scan['pp3file'][0],  outputfolderpath=scan['RToutputfolder'], axis=1)     
            scandf2 = scandf2.append(scan)
    else:
        scan['imagedf'] = scan['imagedf'].apply(RClib.baseimageIsDevimage, axis=1)
        scandf2 = scandf2.append(scan)
        
scandf3=pd.DataFrame()
if config['align']:
    for index,scan in scandf2.iterrows():
        if not scan['imagedf'] is None:    
            scan = RClib.makeImagelist(scan, imagelistname=scan['id'])     
            scan = RClib.createRCproject(scan)
            scan =RClib.readProcessingstateFile(scan)    
            if not RClib.checkProcessingstate(scan,command='commandlist_alignsettings'):
                scan['commandlist_alignsettings']=Path(config['alignmentsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_alignsettings' )
            if not RClib.checkProcessingstate(scan,command='align-rccmdpath'):
                scan['commandlist_align'] = []
                scan['commandlist_align'].append('-add ' + str(scan['list_dev-img_path']))
                
                if config['use_gcps']:
                    scan['commandlist_align'].append('-detectMarkers')
                    if Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file():
                        scan['commandlist_align'].append('-importGroundControlPoints ' + str(config['constantgcps']) + ' ' + config['gcpsparamfile'])
                    if not Path(config['constantgcps']).is_file() and Path(config['gcpsparamfile']).is_file() and scan['gcpsfile']:
                        scan['commandlist_align'].append('-importGroundControlPoints ' + str(scan['gcpsfile'][0]) + ' ' + config['gcpsparamfile'] )
                scan['commandlist_align'].append('-align')
                scan['commandlist_align'].append('-setReconstructionRegionAuto')
                #scan['commandlist_align'].append('-exportReport ' + str(Path(scan['RCoutputfolder'] / Path(scan['id']+'.txt '))) + config['alignmentreport_template'])
                scan['commandlist_align'].append('-save')
                scan['commandlist_align'].append('-quit')
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_align', rccmdpathfield='align-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='align-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
            #scan['processingstatus'].append({'alignment_report': Path(scan['RCoutputfolder'] / Path(scan['id']+'.html '))})
            scandf3 = scandf3.append(scan)
else:
    scandf3 = scandf
#pd.write_csv(Path(config['workspace']) / 'rcprocessingdf.csv', sep=';')
#scandf2.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

C:\Users\gilgamesh\AppData\Local\Temp\ipykernel_7272\4161106697.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05983.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\02_TIF\DSC05983.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05984.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\02_TIF\DSC05984.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05985.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\02_TIF\DSC05985.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05986.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\02_TIF\DSC05986.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05987.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\02_TIF\DSC05987.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05988.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\02_TIF\DSC05988.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05989.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\02_TIF\DSC05989.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_1to2m\DSC05990.dng
Expect 

C:\Users\gilgamesh\AppData\Local\Temp\ipykernel_7272\4161106697.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06062.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\02_TIF\DSC06062.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06063.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\02_TIF\DSC06063.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06064.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\02_TIF\DSC06064.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06065.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\02_TIF\DSC06065.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06066.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\02_TIF\DSC06066.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06067.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\02_TIF\DSC06067.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06068.dng
Expect file:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\02_TIF\DSC06068.tif
Inputfile:  E:\WES_paleoenvi\URUK26\URUK26_2to3m\DSC06069.dng
Expect 

C:\Users\gilgamesh\AppData\Local\Temp\ipykernel_7272\4161106697.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf2 = scandf2.append(scan)


...

C:\Users\gilgamesh\AppData\Local\Temp\ipykernel_7272\4161106697.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


E:\WES_paleoenvi\URUK26\URUK26_5to6m\03_RC\commandlist_align_URUK26_5to6m.rccmd


C:\Users\gilgamesh\AppData\Local\Temp\ipykernel_7272\4161106697.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


E:\WES_paleoenvi\URUK26\URUK26_6to7m\03_RC\commandlist_align_URUK26_6to7m.rccmd


C:\Users\gilgamesh\AppData\Local\Temp\ipykernel_7272\4161106697.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  scandf3 = scandf3.append(scan)


In [12]:
%load_ext autoreload
%autoreload 2
scandf2= scandf2.apply(RClib.defineResultOutput, foldername='04_GEO', axis=1)
#while 'reconstruct-rccmdpath'.isin(scandf3['processingstate'])] and 'model-rccmdpath'.notin(scandf3['processingstate']) :
    #tobemodeleddf = scandf3['reconstruct-rccmdpath'.isin(scandf3['processingstate']) and 'model-rccmdpath'.notin(scandf3['processingstate']) ]
scandf3 = pd.DataFrame()
if config['reconstruct']:
    for index,scan in scandf2.iterrows():
        if 'constantrcbox' in config.keys() and Path(config['constantrcbox']).is_file():
            scan['rcboxpath'] = Path(config['constantrcbox'])
        else:
            scan['rcboxpath'] = Path(scan['RCoutputfolder'] / 'base.rcbox')
        scan =RClib.readProcessingstateFile(scan)    
        if RClib.checkProcessingstate(scan,command='align-rccmdpath') and scan['rcboxpath'].is_file():
            if not RClib.checkProcessingstate(scan,command='commandlist_reconstructsettings'):
                scan['commandlist_reconstructsettings']=Path(config['reconstructsettings'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_reconstructsettings')
            if not RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'): 
                scan['commandlist_reconstruct'] = []
                scan['commandlist_reconstruct'].append('-setReconstructionRegion ' + str(scan['rcboxpath']))  
                scan['commandlist_reconstruct'].append('-calculateNormalModel')  
                scan['commandlist_reconstruct'].append('-renameSelectedModel ' + scan['id'] + '_highpoly')    
                scan['commandlist_reconstruct'].append('-save')
                scan['commandlist_reconstruct'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_reconstruct', rccmdpathfield='reconstruct-rccmdpath')
                print('Starting reconstruction for  ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='reconstruct-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf3 = scandf3.append(scan)
else:
    scandf3 = scandf2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
scandf4 = pd.DataFrame()
if config['texture']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_texturingsettings'):
                scan['commandlist_texturingsettings']=Path(config['texturingsettings'])
                print('Texturing settings set for ', scan['id'])
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='commandlist_texturingsettings')
            if not RClib.checkProcessingstate(scan,command='texturing-rccmdpath'):
                scan['commandlist_texturing'] = []
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly') 
                scan['commandlist_texturing'].append('-smooth') 
                scan['commandlist_texturing'].append('-cleanModel')
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_highpoly' + '_smoothed')
                scan['commandlist_texturing'].append('-correctColors geometry') 
                scan['commandlist_texturing'].append('-unwrap')  
                scan['commandlist_texturing'].append('-calculateTexture')
                scan['commandlist_texturing'].append('-simplify 10000000')
                scan['commandlist_texturing'].append('-cleanModel')
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_10M')
                scan['commandlist_texturing'].append('-unwrap')
                scan['commandlist_texturing'].append('-reprojectTexture ' + scan['id'] + '_highpoly' + '_smoothed' + ' ' + scan['id'] + '_10M')
                scan['modelspaths'] = {'modelpath_10M': scan['Resultoutputfolder'] / Path(config['exportname_prefix'] + scan['id'] + '_10M.obj')}
                print(scan['modelspaths'])
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_10M ' + str(scan['modelspaths'].get('modelpath_10M')) + ' ' + config['exportsettings1'] )
                scan['commandlist_texturing'].append('-selectModel ' + scan['id'] + '_highpoly' + '_smoothed')   
                scan['commandlist_texturing'].append('-simplify 40000000')
                scan['commandlist_texturing'].append('-cleanModel') 
                scan['commandlist_texturing'].append('-renameSelectedModel ' + scan['id'] + '_40M')
                scan['commandlist_texturing'].append('-unwrap')
                scan['commandlist_texturing'].append('-reprojectTexture ' + scan['id'] + '_highpoly' + '_smoothed' + ' ' + scan['id'] + '_40M')
                scan['modelspaths']['modelpath_40M'] = scan['Resultoutputfolder'] / Path( config['exportname_prefix'] + scan['id'] + '_40M.obj')
                scan['commandlist_texturing'].append('-exportModel ' + scan['id'] + '_40M ' + str(scan['modelspaths'].get('modelpath_40M')) + ' ' + config['exportsettings1'] )           
                scan['commandlist_texturing'].append('-save')
                scan['commandlist_texturing'].append('-quit')  
                scan = RClib.makeRCCMDfromListfield(scan, commandlistfield='commandlist_texturing', rccmdpathfield='texturing-rccmdpath')
                RClib.executeRCCMDuseRCproject(scan, rccmdpathfield='texturing-rccmdpath' )
                RClib.writeProcessingstateFile(scan)
        scandf4 = scandf4.append(scan)
else:
    scandf4=scandf3
scandf4.to_pickle(str(Path(config['workspace']) / 'rcprocessingdf.pkl'))

Texturing settings set for  PG_boat91_test
{'modelpath_10M': WindowsPath('E:/RCprocessing_test/PG_boat91_test/04_GEO/Zed-ext_0000001_PG_boat91_test_10M.obj')}
E:\RCprocessing_test\PG_boat91_test\03_RC\commandlist_texturing_PG_boat91_test.rccmd


PermissionError: [WinError 32] Der Prozess kann nicht auf die Datei zugreifen, da sie von einem anderen Prozess verwendet wird: 'E:\\RCprocessing_test\\PG_boat91_test\\03_RC\\Processingstate_PG_boat91_test.json'

In [ ]:
if config['orthomosaic']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):
            if not RClib.checkProcessingstate(scan,command='commandlist_orthoimage'):
                if 'constantorthobox' in config.keys() and Path(config['constantorthobox']).is_file():
                    scan['orthoboxpath'] = Path(config['constantorthobox'])
                    scan['commandlist_orthoimage'] = []
                    scan['commandlist_orthoimage'].append('-selectModel ' + scan['id'] + '_highpoly') 
                    scan['commandlist_orthoimage'].append('-calculateOrthoProjection ' + scan['orthoboxpath'] ) 
                    scan['commandlist_orthoimage'].append('-exportOrthoProjection ' + scan['orthoboxpath'] ) 
            

In [ ]:
if config['orthobox2gis']:
    for index,scan in scandf3.iterrows():
        scan = RClib.readProcessingstateFile(scan)
        if RClib.checkProcessingstate(scan,command='reconstruct-rccmdpath'):